In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

import sys, os, gc, types
import time
from subprocess import check_output
import tensorflow as tf

In [2]:
root_paths = [
    "/data/kaggle-wikipedia/",
    "/Users/jiayou/Dropbox/JuanCode/Kaggle/Wikipedia/data/",
    "/Users/jiayou/Dropbox/Documents/JuanCode/Kaggle/Wikipedia/data/"
]
root = None
for p in root_paths:
    if os.path.exists(p):
        root = p
        break
print(check_output(["ls", root]).decode("utf8"))

key_1.csv
sample_submission_1.csv
spider.txt
train_1.csv



In [3]:
train = pd.read_csv(root + 'train_1.csv')
train.fillna(0, inplace = True)

In [4]:
from collections import deque
from datetime import datetime
import logging
import os, imp
import pprint as pp

import numpy as np
import tensorflow as tf

def shape(tensor, dim=None):
    """Get tensor shape/dimension as list/int"""
    if dim is None:
        return tensor.shape.as_list()
    else:
        return tensor.shape.as_list()[dim]

class TFBaseModel(object):

    """Interface containing some boilerplate code for training tensorflow models.
    Subclassing models must implement self.calculate_loss(), which returns a tensor for the batch loss.
    Code for the training loop, parameter updates, checkpointing, and inference are implemented here and
    subclasses are mainly responsible for building the computational graph beginning with the placeholders
    and ending with the loss tensor.
    Args:
        reader: Class with attributes train_batch_generator, val_batch_generator, and test_batch_generator
            that yield dictionaries mapping tf.placeholder names (as strings) to batch data (numpy arrays).
        batch_size: Minibatch size.
        learning_rate: Learning rate.
        optimizer: 'rms' for RMSProp, 'adam' for Adam, 'sgd' for SGD
        grad_clip: Clip gradients elementwise to have norm at most equal to grad_clip.
        regularization_constant:  Regularization constant applied to all trainable parameters.
        early_stopping_steps:  Number of steps to continue training after validation loss has
            stopped decreasing.
        warm_start_init_step:  If nonzero, model will resume training a restored model beginning
            at warm_start_init_step.
        num_restarts:  After validation loss plateaus, the best checkpoint will be restored and the
            learning rate will be halved.  This process will repeat num_restarts times.
        enable_parameter_averaging:  If true, model saves exponential weighted averages of parameters
            to separate checkpoint file.
        min_steps_to_checkpoint:  Model only saves after min_steps_to_checkpoint training steps
            have passed.
        log_interval:  Train and validation accuracies are logged every log_interval training steps.
        loss_averaging_window:  Train/validation losses are averaged over the last loss_averaging_window
            training steps.
        num_validation_batches:  Number of batches to be used in validation evaluation at each step.
        log_dir: Directory where logs are written.
        checkpoint_dir: Directory where checkpoints are saved.
        prediction_dir: Directory where predictions/outputs are saved.
    """

    def __init__(
        self,
        reader,
        batch_size=128,
        num_training_steps=20000,
        learning_rate=.01,
        optimizer='adam',
        grad_clip=5,
        regularization_constant=0.0,
        early_stopping_steps=3000,
        warm_start_init_step=0,
        num_restarts=None,
        enable_parameter_averaging=False,
        min_steps_to_checkpoint=100,
        log_interval=20,
        loss_averaging_window=100,
        num_validation_batches=1,
        work_dir='tf-data',
        name='nn'
    ):

        self.reader = reader
        self.batch_size = batch_size
        self.num_training_steps = num_training_steps
        self.learning_rate = learning_rate
        self.optimizer = optimizer
        self.grad_clip = grad_clip
        self.regularization_constant = regularization_constant
        self.warm_start_init_step = warm_start_init_step
        self.early_stopping_steps = early_stopping_steps if early_stopping_steps is not None else np.inf
        self.enable_parameter_averaging = enable_parameter_averaging
        self.num_restarts = num_restarts
        self.min_steps_to_checkpoint = min_steps_to_checkpoint
        self.log_interval = log_interval
        self.num_validation_batches = num_validation_batches
        self.loss_averaging_window = loss_averaging_window
        self.name = name

        self.log_dir = os.path.join(work_dir, 'logs')
        self.prediction_dir = os.path.join(work_dir, 'predictions')
        self.checkpoint_dir = os.path.join(work_dir, 'checkpoints')
        if self.enable_parameter_averaging:
            self.checkpoint_dir_averaged = os.path.join(work_dir, 'checkpoints-avg')

        self.init_logging(self.log_dir)
        self.logger.info('\nNetwork hyper-parameters:\n{}'.format(pp.pformat(self.__dict__)))
        self.reader.describe(self.logger)

        self.graph = self.build_graph()
        self.session = tf.Session(graph=self.graph)
        print('Built graph')

    def calculate_loss(self):
        raise NotImplementedError('subclass must implement this')

    def fit(self):
        with self.session.as_default():

            if self.warm_start_init_step:
                self.restore(self.warm_start_init_step)
                step = self.warm_start_init_step
            else:
                self.session.run(self.init)
                step = 0

            train_generator = self.reader.train_batch_generator(self.batch_size)
            val_generator = self.reader.val_batch_generator(self.num_validation_batches*self.batch_size)

            train_loss_history = deque(maxlen=self.loss_averaging_window)
            val_loss_history = deque(maxlen=self.loss_averaging_window)

            best_validation_loss, best_validation_tstep = float('inf'), 0
            restarts = 0

            while step < self.num_training_steps:

                # validation evaluation
                val_batch_df = next(val_generator)
                val_feed_dict = {
                    getattr(self, placeholder_name, None): data
                    for placeholder_name, data in val_batch_df.items() if hasattr(self, placeholder_name)
                }

                val_feed_dict.update({self.learning_rate_var: self.learning_rate})
                [val_loss] = self.session.run(
                    fetches=[self.loss],
                    feed_dict=val_feed_dict
                )
                val_loss_history.append(val_loss)

                # train step
                train_batch_df = next(train_generator)
                train_feed_dict = {
                    getattr(self, placeholder_name, None): data
                    for placeholder_name, data in train_batch_df.items() if hasattr(self, placeholder_name)
                }

                train_feed_dict.update({self.learning_rate_var: self.learning_rate})
                train_loss, _ = self.session.run(
                    fetches=[self.loss, self.step],
                    feed_dict=train_feed_dict
                )
                train_loss_history.append(train_loss)

                if step % self.log_interval == 0 or step + 1 == self.num_training_steps:
                    avg_train_loss = sum(train_loss_history) / len(train_loss_history)
                    avg_val_loss = sum(val_loss_history) / len(val_loss_history)
                    metric_log = (
                        "[[step {:>8}]]     "
                        "[[train]]     loss: {:<12}     "
                        "[[val]]     loss: {:<12}"
                    ).format(step, round(avg_train_loss, 8), round(avg_val_loss, 8))
                    self.logger.info(metric_log)

                    if avg_val_loss < best_validation_loss:
                        best_validation_loss = avg_val_loss
                        best_validation_tstep = step
                        if step > self.min_steps_to_checkpoint:
                            self.save(step)
                            if self.enable_parameter_averaging:
                                self.save(step, averaged=True)

                    if step - best_validation_tstep >= self.early_stopping_steps:

                        if self.num_restarts is None or restarts >= self.num_restarts:
                            self.logger.info('Early stopping')
                            break

                        if restarts < self.num_restarts:
                            self.logger.info('')
                            self.restore(best_validation_tstep)
                            self.learning_rate /= 2.0
#                             self.early_stopping_steps /= 2
                            step = best_validation_tstep
                            restarts += 1
                            self.logger.info(
                                'Half learning rate to {} and restore step {}'.format(self.learning_rate, step)
                            )

                step += 1

            if step <= self.min_steps_to_checkpoint:
                best_validation_tstep = step
                self.save(step)
                if self.enable_parameter_averaging:
                    self.save(step, averaged=True)

            self.logger.info('Training ended')
            self.logger.info(
                'Best validation loss of {} at training step {}'.format(
                    round(best_validation_loss, 8), 
                    best_validation_tstep
                )
            )

    def predict(self, chunk_size=2048):
        if not os.path.isdir(self.prediction_dir):
            os.makedirs(self.prediction_dir)

        preds = {}
        if hasattr(self, 'prediction_tensors'):
            prediction_dict = {tensor_name: [] for tensor_name in self.prediction_tensors}

            test_generator = self.reader.test_batch_generator(chunk_size)
            for i, test_batch_df in enumerate(test_generator):
                test_feed_dict = {
                    getattr(self, placeholder_name, None): data
                    for placeholder_name, data in test_batch_df.items() if hasattr(self, placeholder_name)
                }

                tensor_names, tf_tensors = zip(*self.prediction_tensors.items())
                np_tensors = self.session.run(
                    fetches=tf_tensors,
                    feed_dict=test_feed_dict
                )
                for tensor_name, tensor in zip(tensor_names, np_tensors):
                    prediction_dict[tensor_name].append(tensor)

            for tensor_name, tensor in prediction_dict.items():
                np_tensor = np.concatenate(tensor, 0)
                preds[tensor_name] = np_tensor
                save_file = os.path.join(self.prediction_dir, '{}.npy'.format(tensor_name))
                self.logger.info('Saving {} with shape {} to {}'.format(tensor_name, np_tensor.shape, save_file))
                np.save(save_file, np_tensor)

        if hasattr(self, 'parameter_tensors'):
            for tensor_name, tensor in self.parameter_tensors.items():
                np_tensor = tensor.eval(self.session)

                save_file = os.path.join(self.prediction_dir, '{}.npy'.format(tensor_name))
                self.logger.info('Saving {} with shape {} to {}'.format(tensor_name, np_tensor.shape, save_file))
                np.save(save_file, np_tensor)

        return preds

    def save(self, step, averaged=False):
        saver = self.saver_averaged if averaged else self.saver
        checkpoint_dir = self.checkpoint_dir_averaged if averaged else self.checkpoint_dir
        if not os.path.isdir(checkpoint_dir):
            self.logger.info('creating checkpoint directory {}'.format(checkpoint_dir))
            os.mkdir(checkpoint_dir)

        model_path = os.path.join(checkpoint_dir, 'model')
        self.logger.info('saving model to {}'.format(model_path))
        saver.save(self.session, model_path, global_step=step)

    def restore(self, step=None, averaged=False):
        saver = self.saver_averaged if averaged else self.saver
        checkpoint_dir = self.checkpoint_dir_averaged if averaged else self.checkpoint_dir
        if not step:
            model_path = tf.train.latest_checkpoint(checkpoint_dir)
            self.logger.info('Restoring model parameters from {}'.format(model_path))
            saver.restore(self.session, model_path)
        else:
            model_path = os.path.join(
                checkpoint_dir, 'model{}-{}'.format('_avg' if averaged else '', step)
            )
            self.logger.info('Restoring model from {}'.format(model_path))
            saver.restore(self.session, model_path)

    def init_logging(self, log_dir):
        if not os.path.isdir(log_dir):
            os.makedirs(log_dir)
        
        logger = logging.getLogger(
            '{}.{}'.format(
                type(self).__name__, 
                datetime.now().strftime('%Y-%m-%d.%H-%M-%S.%f')
            )
        )
        logger.setLevel(logging.INFO)
        fmtr = logging.Formatter(
            fmt='[[%(asctime)s]] %(message)s',
            datefmt='%m/%d/%Y %I:%M:%S %p'
        )

        h = logging.StreamHandler(stream=sys.stdout)
        h.setFormatter(fmtr)
        logger.addHandler(h)
        
        date_str = datetime.now().strftime('%Y-%m-%d_%H-%M')
        log_file = 'log.{}.{}.{}.txt'.format(type(self).__name__, self.name, date_str)
        h = logging.FileHandler(filename=os.path.join(log_dir, log_file))
        h.setFormatter(fmtr)
        logger.addHandler(h)
        
        self.logger = logger

    def update_parameters(self, loss):
        self.global_step = tf.Variable(0, trainable=False)
        self.learning_rate_var = tf.Variable(0.0, trainable=False)

        if self.regularization_constant != 0:
            l2_norm = tf.reduce_sum([tf.sqrt(tf.reduce_sum(tf.square(param))) for param in tf.trainable_variables()])
            loss = loss + self.regularization_constant*l2_norm

        optimizer = self.get_optimizer(self.learning_rate_var)
        grads = optimizer.compute_gradients(loss)
        clipped = [(tf.clip_by_value(g, -self.grad_clip, self.grad_clip), v_) for g, v_ in grads]

        step = optimizer.apply_gradients(clipped, global_step=self.global_step)

        if self.enable_parameter_averaging:
            maintain_averages_op = self.ema.apply(tf.trainable_variables())
            with tf.control_dependencies([step]):
                self.step = tf.group(maintain_averages_op)
        else:
            self.step = step

    def log_parameters(self):
        self.logger.info(
            (
                '\n\n'
                'All parameters:\n{}\n'
                'Trainable parameters:\n{}\n'
                'Trainable parameter count: {}\n'
            ).format(
                pp.pformat([(var.name, shape(var)) for var in tf.global_variables()]),
                pp.pformat([(var.name, shape(var)) for var in tf.trainable_variables()]),
                str(np.sum(np.prod(shape(var)) for var in tf.trainable_variables())),
            )
        )

    def get_optimizer(self, learning_rate):
        if self.optimizer == 'adam':
            return tf.train.AdamOptimizer(learning_rate)
        elif self.optimizer == 'gd':
            return tf.train.GradientDescentOptimizer(learning_rate)
        elif self.optimizer == 'rms':
            return tf.train.RMSPropOptimizer(learning_rate, decay=0.95, momentum=0.9)
        else:
            assert False, 'optimizer must be adam, gd, or rms'

    def build_graph(self):
        with tf.Graph().as_default() as graph:
            self.ema = tf.train.ExponentialMovingAverage(decay=0.995)

            self.loss = self.calculate_loss()
            self.update_parameters(self.loss)
            self.log_parameters()

            self.saver = tf.train.Saver(max_to_keep=1)
            if self.enable_parameter_averaging:
                self.saver_averaged = tf.train.Saver(self.ema.variables_to_restore(), max_to_keep=1)

            self.init = tf.global_variables_initializer()

            return graph

In [10]:
class DataReader():
    def __init__(self, data, seed=923):
        self.data = data.iloc[:,1:].values
        self.seed = seed
        self.days = self.data.shape[1]

        
    def describe(self, logger):
        logger.info('')
        logger.info('Data dimensions:')
        logger.info('    [[data]] {}'.format(self.data.shape))
        logger.info('Split seed = {}'.format(self.seed))
        logger.info('')

    def train_batch_generator(self, batch_size):
        return self.batch_generator(
            self.data,
            batch_size=batch_size,
            given_days=self.days - 62,
            no_loss_days=30,
            days=self.days - 62,
        )

    def val_batch_generator(self, batch_size):
        return self.batch_generator(
            self.data,
            batch_size=batch_size,
            given_days=self.days - 62,
            no_loss_days=self.days - 62,
            days=self.days,
        )

    def test_batch_generator(self, batch_size):
        pass

    def batch_generator(self, data, batch_size, given_days, no_loss_days, days):
        while True:
            batch = {}
            idx = np.random.randint(0, data.shape[0], [batch_size])
            batch['data'] = data[idx, :]
            batch['given_days'] = given_days
            batch['no_loss_days'] = no_loss_days
            batch['days'] = days
            yield batch

In [30]:
class WikiRNN(TFBaseModel):

    def __init__(self, state_size, keep_prob=1, **kwargs):
        self.state_size = state_size
        self.keep_prob = keep_prob
        super(type(self), self).__init__(**kwargs)

    def calculate_loss(self):
        self.data = tf.placeholder(tf.float32, name='data')
        self.given_days = tf.placeholder(tf.int32, name='given_days')
        self.no_loss_days = tf.placeholder(tf.int32, name='no_loss_days')
        self.days = tf.placeholder(tf.int32, name='days')
        
        cell = tf.contrib.rnn.DropoutWrapper(
            tf.contrib.rnn.LSTMCell(
                self.state_size
            ),
            output_keep_prob=self.keep_prob
        )
        
        # [batch_size, state_size]
        state = cell.zero_state(tf.shape(self.data)[0], dtype=tf.float32)
        # [batch_size, 1]
        last_output = tf.zeros([tf.shape(self.data)[0], 1], dtype=tf.float32)
        
        loss = tf.constant(0, dtype=tf.float32)
        step = tf.constant(0, dtype=tf.int32)
        
        def cond(last_output, state, loss, step):
            return step < self.days
        
        def body(last_output, state, loss, step):
            output, state = cell(last_output, state)
            output = tf.layers.dense(
                output,
                1,
                activation=tf.nn.relu,
                name='dense-top'
            )
            
            last_output = tf.cond(
                step < self.given_days,
                lambda: tf.expand_dims(self.data[:,step], 1),
                lambda: output,
            )
            last_output.set_shape([None, 1])
            last_output = tf.round(last_output)
            
            loss = tf.cond(
                step >= self.no_loss_days,
                lambda: \
                    loss + \
                    tf.reduce_mean(2 * tf.abs(self.data[:,step] - output) \
                    / tf.maximum(1e-8, self.data[:,step] + output)),
                lambda: loss
            )
            loss.set_shape([])
            
            return (last_output, state, loss, step + 1)
        
        _, _, loss, _ = tf.while_loop(
            cond=cond,
            body=body,
            loop_vars=(last_output, state, loss, step)
        )
        
        return loss / tf.cast(self.days - self.no_loss_days, tf.float32) * 100


In [12]:
reader = DataReader(train, seed=923)

In [ ]:
for i in range(1):
    # 100 steps make an epoch
    nn = WikiRNN(
        name='v0',
        reader=reader,
        work_dir='./tf-data',
        optimizer='adam',
        learning_rate=1.0001,
        batch_size=1,
        num_validation_batches=1,
        num_training_steps=10000,
        early_stopping_steps=300,
        num_restarts=3,
        warm_start_init_step=0,
        regularization_constant=0.0,
        enable_parameter_averaging=False,
        min_steps_to_checkpoint=0,
        loss_averaging_window=1,
        log_interval=1,
        
        state_size=10, 
        keep_prob=1
    )
    nn.fit()
#     nn.restore()
#     preds = nn.predict()

[[08/28/2017 03:24:09 PM]] 
Network hyper-parameters:
{'batch_size': 1,
 'checkpoint_dir': './tf-data/checkpoints',
 'early_stopping_steps': 300,
 'enable_parameter_averaging': False,
 'grad_clip': 5,
 'keep_prob': 1,
 'learning_rate': 1.0001,
 'log_dir': './tf-data/logs',
 'log_interval': 1,
 'logger': <Logger WikiRNN.2017-08-28.15-24-09.406871 (INFO)>,
 'loss_averaging_window': 1,
 'min_steps_to_checkpoint': 0,
 'name': 'v0',
 'num_restarts': 3,
 'num_training_steps': 10000,
 'num_validation_batches': 1,
 'optimizer': 'adam',
 'prediction_dir': './tf-data/predictions',
 'reader': <__main__.DataReader object at 0x11f70e438>,
 'regularization_constant': 0.0,
 'state_size': 10,
 'warm_start_init_step': 0}
[[08/28/2017 03:24:09 PM]] 
[[08/28/2017 03:24:09 PM]] Data dimensions:
[[08/28/2017 03:24:09 PM]]     [[data]] (145063, 550)
[[08/28/2017 03:24:09 PM]] Split seed = 923
[[08/28/2017 03:24:09 PM]] 
[[08/28/2017 03:24:10 PM]] 

All parameters:
[('lstm_cell/weights:0', [11, 40]),
 ('lstm